In [7]:
# imports
import csv
import os
import pathlib
from collections import namedtuple
from time import time
from typing import List, Tuple, Optional
from random import randint, seed

import networkx as nx
import numpy as np
import torch
import torch_geometric.utils as tutils
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset
from tqdm import tqdm_notebook as tqdm

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from grakel.kernels import WeisfeilerLehman, VertexHistogram, ShortestPath
from grakel import GraphKernel
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit

from utils import load_singleton_graphs_from_TUDataset

import warnings
warnings.filterwarnings('ignore')

In [23]:
# Parameters
datasets = ['AIDS', 'BZR', 'BZR_MD', 'COX2', 'COX2_MD', 'DHFR', 'DHFR_MD', 'ER_MD', 'MUTAG', 'Mutagenicity', 'NCI1', 'NCI109', 'PTC_FM', 'PTC_FR', 'PTC_MM', 'PTC_MR',
            'DD', 'ENZYMES', 'KKI', 'OHSU', 'Peking_1', 'PROTEINS_full',
            'MSRC_9', 'MSRC_21', 
            ]
            #'COLLAB', 'REDDIT-BINARY', 'REDDIT-MULTI-5K', 'REDDIT-MULTI-12K']
# 'NCI-H23H''AIDS', 
seed(42)
seeds = [randint(0, 10000) for _ in range(10)]

In [24]:
# CONSTANTS
NODE_ATTRIBUTE = 'x'

CLF_METHODS = {
    'knn': (KNeighborsClassifier, {'kneighborsclassifier__n_neighbors': [3, 5, 7, 9, 11]}),
    'rbf': (SVC, {#'svc__gamma': np.logspace(-6, 2, 9),
                  'svc__C': np.logspace(-2, 2, 5)})
}


In [25]:
from torch_geometric.utils import degree

def load_special_singleton(root: str,
                                         dataset: str,
                                         node_attr: str = 'x') -> Tuple[List[np.ndarray], np.ndarray]:
    """
    Use the Pytorch Geometric (PyG) loader to download the graph dataset from the TUDataset repo.
    The raw graphs from PyG are saved in `root`.
    
    A singleton graph is created by summing all the node attributes `x` of the original graph.
    The corresponding class of each graph is also retrieved from TUDataset dataset.
    
    Args:
        root: Path where to save the raw graph dataset
        name_dataset: Name of the graph dataset to load
    Returns:
        List of the loaded `np.ndarray` singleton graphs and `np.ndarray` of the corresponding class of each graph
    """
    dataset = TUDataset(root=root, name=dataset)

    tmp_graph = dataset[0]
    is_graph_labelled = node_attr in tmp_graph.keys

    # Convert the PyG graphs into singleton graphs
    graphs = []
    graph_labels = []
    for graph in tqdm(dataset, desc='Convert graph to singleton'):
        
        if not is_graph_labelled:
            # Create graph with dummy node vector
            graph = Data(x=torch.tensor(np.ones((graph.num_nodes, 1))),
                         y=graph.y,
                         edge_index=graph.edge_index)
        degrees = degree(graph.edge_index[0], graph.num_nodes)
        node_feature = torch.mul(graph.x, degrees.view(-1, 1))
        
        graphs.append(np.array(node_feature.sum(axis=0)))
        graph_labels.append(int(graph.y))

    graph_cls = np.array(graph_labels)

    return graphs, graph_cls

In [26]:
def distinct_graphs(graphs):
    unique_graphs = []
    for graph in graphs:
        
        if not any(np.array_equal(graph, g_) for g_ in unique_graphs):
            unique_graphs.append(graph)
            # unique_graphs.append(graph)
        
            
    return unique_graphs

In [30]:
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from collections import Counter
import matplotlib.pyplot as plt


n_jobs = 10
cv = 5
verbose = False

results = {}
folder = './predictions'

for dataset in datasets:
    print(dataset)

    graphs, labels = load_singleton_graphs_from_TUDataset(root=os.path.join('root', dataset),
                                                          dataset=dataset,
                                                          node_attr=NODE_ATTRIBUTE)
#     graphs, labels = load_special_singleton(root=os.path.join('root', dataset),
#                                                           dataset=dataset,
#                                                           node_attr=NODE_ATTRIBUTE)
    # print(f'Distinct graphs {len(distinct_graphs(graphs))}/{len(graphs)}')
    
    print("class proportions", np.unique(labels, return_counts=True)[1] / len(labels))
    # plt.hist(labels)
    
    for clf_name, clf in CLF_METHODS.items():
        clf_method, param_grid = clf

        f1_scores = []
        accuracies = []
        precisions, recalls = [], []
        balanced_accuracies = []
        aucrocs = []
        

        for c_seed in seeds:
            
            G_train, G_test, y_train, y_test = train_test_split(graphs,
                                                                labels,
                                                                test_size=0.2,
                                                                random_state=c_seed,
                                                                stratify=labels)
            pipe_clf = make_pipeline(StandardScaler(),
                                     clf_method())
            clf = GridSearchCV(estimator=pipe_clf,
                                param_grid=param_grid,
                                n_jobs=n_jobs,
                                cv=cv,
                               scoring='f1_micro',
                                verbose=int(verbose)*3)
            
            
            clf.fit(G_train, y_train)
            y_predictions = clf.predict(G_test)
            
            
            
            accuracies.append(accuracy_score(y_test, y_predictions))
            balanced_accuracies.append(balanced_accuracy_score(y_test, y_predictions))
            average = 'binary' if len(set(labels)) <= 2 else 'macro'
            f1_scores.append(f1_score(y_test, y_predictions, average=average))
            precisions.append(precision_score(y_test, y_predictions))
            recalls.append(recall_score(y_test, y_predictions))
            aucrocs.append(roc_auc_score(y_test, y_predictions, multi_class='ovr'))
        
        print(f'{dataset}, {clf_name} (acc): {np.mean(accuracies)*100:.2f} +- {np.std(accuracies)*100:.2f}')
        print(f'{dataset}, {clf_name} (balanced acc): {np.mean(balanced_accuracies)*100:.2f} +- {np.std(balanced_accuracies)*100:.2f}')
        print(f'{dataset}, {clf_name} (f1): {np.mean(f1_scores)*100:.2f} +- {np.std(f1_scores)*100:.2f}')
        print(f'{dataset}, {clf_name} (precision): {np.mean(precisions)*100:.2f} +- {np.std(precisions)*100:.2f}')
        print(f'{dataset}, {clf_name} (recall): {np.mean(recalls)*100:.2f} +- {np.std(recalls)*100:.2f}')
        print(f'{dataset}, {clf_name} (AUCROC): {np.mean(aucrocs)*100:.2f} +- {np.std(aucrocs)*100:.2f}')
        print('*' * 50)
            
    print('='*60)
    
  

AIDS


Convert graph to singleton: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 6833.23it/s]


class proportions [0.2 0.8]
AIDS, knn (acc): 98.15 +- 0.65
AIDS, knn (balanced acc): 96.03 +- 1.33
AIDS, knn (f1): 98.85 +- 0.40
AIDS, knn (precision): 98.15 +- 0.61
AIDS, knn (recall): 99.56 +- 0.32
AIDS, knn (AUCROC): 96.03 +- 1.33
**************************************************
AIDS, rbf (acc): 99.05 +- 0.40
AIDS, rbf (balanced acc): 97.91 +- 0.89
AIDS, rbf (f1): 99.41 +- 0.25
AIDS, rbf (precision): 99.01 +- 0.43
AIDS, rbf (recall): 99.81 +- 0.21
AIDS, rbf (AUCROC): 97.91 +- 0.89
**************************************************
BZR


Convert graph to singleton: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 405/405 [00:00<00:00, 17653.71it/s]

class proportions [0.78765432 0.21234568]


...

class proportions [0.58658744 0.41341256]


DD, knn (acc): 77.63 +- 2.34
DD, knn (balanced acc): 76.42 +- 2.36
DD, knn (f1): 71.99 +- 2.86
DD, knn (precision): 75.11 +- 3.75
DD, knn (recall): 69.29 +- 3.80
DD, knn (AUCROC): 76.42 +- 2.36
**************************************************
DD, rbf (acc): 78.18 +- 1.94
DD, rbf (balanced acc): 77.36 +- 2.05
DD, rbf (f1): 73.38 +- 2.52
DD, rbf (precision): 74.46 +- 3.22
DD, rbf (recall): 72.55 +- 4.41
DD, rbf (AUCROC): 77.36 +- 2.05
**************************************************
ENZYMES


Convert graph to singleton: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 18112.47it/s]

class proportions [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].